Korzystając z pomysłu zaczerpniętego z tego artykułu - https://towardsdatascience.com/time-series-analysis-in-python-an-introduction-70d5a5b1d52a chciałem zobaczyć czy da się skorelować/porównać zmiany w kursach akcji, dla przykładu, FB i poszukiwania słowa "facebook" w wyszukiwarce google.

1. Do wyciągnięcia danych z Google Trends, z użyciem nioficjalnego API, użyjemy biblioteki Pytrends (https://pypi.org/project/pytrends)

In [ ]:
from pytrends.request import TrendReq

pytrend = TrendReq()
pytrend.build_payload(kw_list = ['Facebook'], timeframe = '2010-01-01 2020-03-31')
df = pytrend.interest_over_time()
df = df.reset_index().drop('isPartial', 1)
df.head(100)

df.plot(x='date', y='Facebook', kind ='line')

2. Używając biblioteki fbrophet (https://facebook.github.io/prophet/) budujemy model dla powyższej ramki danych i wyciągamy z niego "changepoints", czyli daty gdy następowła zmiana trendu.
Korelacja mogłaby byc lepsza...

In [ ]:
import matplotlib.pyplot as plt
import fbprophet

# Prophet wymaga zmainy nazwy kolumn na ds (Data) i y (wartość)
df1 = df.rename(columns={'date': 'ds', 'Facebook': 'y'})

# Dopasowanie modelu Prophet
fb_prophet = fbprophet.Prophet(changepoint_prior_scale=0.3, n_changepoints=15)
fb_prophet.fit(df1)

# Wyciągnięcie changepoints
fb_prophet.changepoints[:10]
fb_changepoints = fb_prophet.changepoints

# Wykres trendów wyszukiwań z nałżonymi changepoints
plt.plot(df['date'], df['Facebook'], label = 'Searches')

plt.vlines(fb_changepoints, ymin = 0, ymax= 100, colors = 'r', linewidth=0.6,
           linestyles = 'dashed', label = 'Changepoints')
 
plt.ylabel('Relative Search Freq'); plt.legend()
plt.title('FB Search Terms and Changepoints');

3. Zmiany trendu ilości wyszukiwań nałożymy na wykres ceny akcji (do dopracowania, quandl nie ściąga całego okresu...)

In [ ]:
import quandl

quandl.ApiConfig.api_key = 'jJLD2gkVUNrszRRebz2U'

data = quandl.get("WIKI/FB", start_date="2010-01-01", end_date="2020-03-31", collapse="monthly")
data1 = data.reset_index()
data1 = data1[['Date','Close']]

plt.plot(data1['Date'], data1['Close'], label = 'Closing price')

# Plot the changepoints
plt.vlines(fb_changepoints, ymin = 0, ymax= 200, colors = 'r', linewidth=0.6,
           linestyles = 'dashed', label = 'Search changepoints')
 
# Formatting of plot
plt.ylabel('Relative Search Freq'); plt.legend()
plt.title('FB Search Terms and Changepoints');